In [1]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
import pickle
import re

In [2]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [3]:
politicians_party = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']

In [4]:
def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return t
        
    return topic

In [194]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-5-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
df['Topic'] = df['Topic'].apply(lambda x: preprocess(x))
print(df['Topic'].unique())
# df = df.loc[df['Topic'] == 'Parliament, government and politics']
# df = df.loc[df['Topic'] == 'European Union']
# df = df.loc[df['Topic'] == 'Crime, civil law, justice and rights']
# df = df.loc[df['Topic'] == 'Health services and medicine']
# df = df.loc[df['Topic'] == 'International affairs']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2']
df.columns = cols
df = df.drop(['speaker'], axis=1)
df.head()

['admin' 'Employment and training' 'Others'
 'Crime, civil law, justice and rights' 'Defence' 'International affairs'
 'Agriculture, animals, food and rural affairs'
 'Health services and medicine' 'Communities and families'
 'Parliament, government and politics' 'Culture, media and sport'
 'Economy and finance' 'Asylum, immigration and nationality'
 'European Union' 'Housing and planning' 'Energy and environment'
 'Transport' 'Education' 'Business, industry and consumers'
 'Social security and pensions' 'Science and technology' 'Social services']


,date,name,Party,Topic,transcript,party2
19003,2015-06-01,Kevin Brennan,Labour,International affairs,The Secretary of State mentioned in his preamb...,0.0
19010,2015-06-01,Rehman Chishti,Conservative,International affairs,My right hon. Friend says that we should not l...,1.0
19015,2015-06-01,Peter Bone,Conservative,International affairs,I am grateful to the Foreign Secretary for giv...,1.0
19022,2015-06-01,Mike Gapes,Labour,International affairs,Has the Foreign Secretary seen the recent repo...,0.0
19025,2015-06-01,Jim Cunningham,Labour,International affairs,rose—,0.0


In [195]:
# df['transcript'] = df[['name','transcript']].groupby(['name'])['transcript'].transform(lambda x: ','.join(x))

In [196]:
# df = df.drop_duplicates()

In [197]:
def get_party(row):
    if row['Party'] == 'Conservative' and row['party2'] == 1:
        party = 'leave_conservative'
    elif row['Party'] == 'Labour' and row['party2'] == 0:
        party = 'remain_labour'
    elif row['Party'] == 'Labour' and row['party2'] == 1:
        party = 'leave_labour'
    else:
        party = 'remain_conservative'
        
    return party

df['party'] = df.apply(lambda x: get_party(x), axis=1)

In [198]:
df

,date,name,Party,Topic,transcript,party2,party
19003,2015-06-01,Kevin Brennan,Labour,International affairs,The Secretary of State mentioned in his preamb...,0.0,remain_labour
19010,2015-06-01,Rehman Chishti,Conservative,International affairs,My right hon. Friend says that we should not l...,1.0,leave_conservative
19015,2015-06-01,Peter Bone,Conservative,International affairs,I am grateful to the Foreign Secretary for giv...,1.0,leave_conservative
19022,2015-06-01,Mike Gapes,Labour,International affairs,Has the Foreign Secretary seen the recent repo...,0.0,remain_labour
19025,2015-06-01,Jim Cunningham,Labour,International affairs,rose—,0.0,remain_labour
...,...,...,...,...,...,...,...
35369,2016-06-29,Stephen Twigg,Labour,International affairs,May I welcome what the Secretary of State said...,0.0,remain_labour
35402,2016-06-29,Bob Blackman,Conservative,International affairs,Has my right hon. Friend examined Save a Child...,1.0,leave_conservative
35408,2016-06-29,Andy Slaughter,Labour,International affairs,T3. Given what the Overseas Development Inst...,0.0,remain_labour
35417,2016-06-29,Wes Streeting,Labour,International affairs,Given the support that the Government provide ...,0.0,remain_labour


In [199]:
df['party'].unique()

array(['remain_labour', 'leave_conservative'], dtype=object)

In [200]:
rows = []

for index, row in df.iterrows():
    if row['Party'] == 'Conservative' and row['party2'] == 1:
        party = 'leave_conservative'
        rows.append([row['name'], party])
    elif row['Party'] == 'Labour' and row['party2'] == 0:
        party = 'remain_labour'
        rows.append([row['name'], party])
    
    
politicians_chosen = pd.DataFrame(rows, columns=['full name', 'party'])
politicians_chosen = politicians_chosen.drop_duplicates().reset_index(drop=True)
politicians_chosen['party'] = politicians_chosen['party'].apply(lambda x: 0 if x == 'leave_conservative' else 1)
politicians_chosen


,full name,party
0,Kevin Brennan,1
1,Rehman Chishti,0
2,Peter Bone,0
3,Mike Gapes,1
4,Jim Cunningham,1
...,...,...
104,Angela Rayner,1
105,Rupa Huq,1
106,Kevan Jones,1
107,Nigel Adams,0


In [201]:
politicians_chosen['party'].value_counts()

1    59
0    50
Name: party, dtype: int64

In [202]:
train_df = df.loc[(df.party == 'leave_conservative') | (df.party == 'remain_labour')]
test_df = df.loc[(df.party == 'remain_conservative')]

In [203]:
def get_bigrams(text):
    # To remove stop words
    stopW = stopwords.words('english')

    # To stem
    ps = PorterStemmer()

    # Tokenize

    clean_transcript = text.lower()
    clean_transcript = word_tokenize(clean_transcript)
    # Remove digits
    clean_transcript = [i for i in clean_transcript if not re.match(r'\d+', i)]
    # Remove Stopwords and single characters
    clean_transcript = [i for i in clean_transcript if i not in stopW and len(i) > 1]
    # Stemming
    clean_transcript = [ps.stem(word) for word in clean_transcript]

    bigrams = []
    
    _len = len(clean_transcript)
    
    for i in range(_len-1):
        bigrams.append(clean_transcript[i] + '.' + clean_transcript[i+1])
        
    return bigrams

In [204]:
all_bigrams = []

n = len(train_df)

for index in tqdm(range(n)):
    row = train_df.iloc[index]
    all_bigrams.extend(get_bigrams(row['transcript']))
    all_bigrams = list(set(all_bigrams))
    
len(all_bigrams)

100%|██████████| 1258/1258 [00:07<00:00, 169.58it/s]


59992

In [205]:
with open('all_bigrams.pkl', 'wb') as f:
    pickle.dump(all_bigrams, f)

In [206]:
with open('all_bigrams.pkl', 'rb') as f:
    all_bigrams = pickle.load(f)

In [207]:
politicians_chosen

,full name,party
0,Kevin Brennan,1
1,Rehman Chishti,0
2,Peter Bone,0
3,Mike Gapes,1
4,Jim Cunningham,1
...,...,...
104,Angela Rayner,1
105,Rupa Huq,1
106,Kevan Jones,1
107,Nigel Adams,0


In [208]:
politicians = politicians_chosen['full name'].values

In [209]:
count_bigrams_by_politicians = {bigram : {politician: 0 for politician in politicians} for bigram in all_bigrams}

In [210]:
n = len(train_df)
for index in tqdm(range(n)):
    row = train_df.iloc[index]
    bigrams = get_bigrams(row['transcript'])
    politician = row['name']
    for bigram in bigrams:
        count_bigrams_by_politicians[bigram][politician] += 1


100%|██████████| 1258/1258 [00:04<00:00, 307.12it/s]


In [211]:
with open('count_bigrams_by_politicians.pkl', 'wb') as f:
    pickle.dump(count_bigrams_by_politicians, f)

In [212]:
with open('count_bigrams_by_politicians.pkl', 'rb') as f:
    count_bigrams_by_politicians = pickle.load(f)


In [213]:
bigrams = list(count_bigrams_by_politicians.keys())
for bigram in bigrams:
    summ = sum(count_bigrams_by_politicians[bigram].values())
    if summ < 5:
        count_bigrams_by_politicians.pop(bigram, None)
        print('removed {}'.format(bigram))

removed day.six
removed chairman.way
removed said.feel
removed last.weekend
removed revenues—promis.baghdad—for
removed minist.believ
removed unanim.secondli
removed colleagu.former
removed region.know
removed offend.public
removed record.execut
removed entir.problem
removed ladi.mani
removed situat.heart
removed south.two
removed move.forward
removed emerg.everi
removed member.well
removed law.inconveni
removed peopl.lebanon
removed power.make
removed say.act
removed heard.alreadi
removed figur.accur
removed manifesto.would
removed hous.author
removed fact.isi
removed summari.damn
removed counter.appal
removed well.comprehens
removed increas.diplomat
removed today—seem.fli
removed flow.islam
removed toward.lower
removed hous.oper
removed suggest.aw
removed matter.would
removed threat.man
removed creat.entir
removed popul.understand
removed problem.given
removed other.nation
removed troop.rais
removed fact.ceasefir
removed massiv.territori
removed union.voic
removed polici.set
removed 

removed cancel.say
removed sea.situat
removed latham.buri
removed degre.school
removed develop.primarili
removed due.illicit
removed abolit.vat
removed later.told
removed sort.system
removed labour.manifesto
removed question.place
removed extrem.poor
removed level.support
removed send.children
removed produc.amnesti
removed invas.territories—not
removed supplier—fund.paid
removed mani.hapless
removed sure.agre
removed highlight.grave
removed renzi.told
removed nation.literaci
removed point.opposit
removed around.need
removed oxfam.médecin
removed far.higher
removed one.bystand
removed growth.area
removed current.provid
removed jone.suggest
removed friend.assist
removed predict.secur
removed messag.peopl
removed gain.gain
removed programm.chang
removed yemen.privileg
removed idi.amin
removed mani.problem
removed three.mother
removed back.may
removed part.speech
removed look.variou
removed court.instig
removed king.abdullah
removed involv.bomb
removed chancellor.right
removed commit.syri

removed behaviour.nip
removed vindict.spite
removed repeal.human
removed enough.want
removed hobson.choic
removed resid.netherland
removed region.becom
removed although.case
removed friend.tell
removed report.may
removed may.join
removed ladi.say
removed got.list
removed pledg.pay
removed clear.deterr
removed asian.forc
removed sentenc.judg
removed total.free
removed defenc.time
removed uk—a.respect
removed encourag.everyon
removed duti.work
removed day.day
removed prison.abroad
removed given.mani
removed support.principl
removed want.armi
removed mr.moon
removed well.make
removed reduc.current
removed link.malawi
removed power.state
removed two.crucial
removed see.whether
removed principl.injur
removed fact.condit
removed encourag.promot
removed land.milk
removed month.russia
removed killer.result
removed given.anybodi
removed whether.crimin
removed accus.complet
removed inequ.neg
removed compet.adult
removed imprison.offend
removed need—.hon
removed posit.take
removed grandiloqu.term

removed clear.need
removed t3.minist
removed stifl.heat
removed interven.somewher
removed domiz.refuge
removed made.implic
removed telegraph.report
removed seizur.cardiac
removed critic.oppon
removed make.better
removed refuge.sought
removed respons.million
removed restor.order
removed lead.term
removed member.silenc
removed watch.document
removed think.learn
removed secretari.unabl
removed sort.law
removed rout.poverti
removed convict.violent
removed biggest.ever
removed report.violent
removed relat.secretari
removed receipt.psychiatr
removed constitut.two-term
removed allow.thrive
removed countri.form
removed movement.citizen
removed start.north
removed disagre.disagre
removed aid.guarante
removed fabric.show
removed close.work
removed ye.minist
removed tribut.bruce
removed huge.excit
removed put.onu
removed alien.elit
removed lesson.histori
removed school.syrian
removed prevent.local
removed want.question
removed daesh.give
removed side.want
removed measur.appeas
removed hakimzadeh.

removed comprehens.ground
removed uk.compani
removed care.read
removed afraid.leader
removed think.doubt
removed countri.said
removed month.predict
removed member.support
removed shall.read
removed sunderland.wish
removed politician.back-bench
removed talk.troublemak
removed two.clauses—then
removed accus.presenc
removed reclaim.power
removed remark.invit
removed remain.democrat
removed denmark.cpr
removed thought.conscienc
removed rememb.argument
removed activist.articl
removed despit.effort
removed unaccept.make
removed committe.secur
removed use.saudi
removed reason.encourag
removed concentr.one
removed anybodi.key
removed real.movement
removed yemeni.peopl
removed maintain.consist
removed impos.twist
removed ident.person
removed think.flee
removed headlin.global
removed eu.uk.
removed muster.fraction
removed region.mani
removed firm.view
removed compani.help
removed fall.threshold
removed jordan.end
removed week.first
removed luxuri.confront
removed morn.focus
removed eu.british
re

...

removed member.give
removed disembark.point
removed given.real
removed billion.peopl
removed pearc.secur
removed come.stay
removed war.choic
removed lesson.world
removed terribl.extent
removed offend.part
removed act.parliament
removed lankan.civil
removed proxi.sort
removed diseas.erad
removed extend.oper
removed commun.govern
removed repres.total
removed particularli.poorli
removed deprav.european
removed elect.place
removed brave.pilot
removed seek.accommod
removed would.seen
removed memori.stir
removed bush.junior
removed need.combin
removed countri.use
removed think.variou
removed us.implement
removed tackl.fundament
removed one.strike
removed world.play
removed unit.us
removed minut.today
removed senat.congress
removed rise.hon
removed hand.arm
removed school.well
removed fuss.standard
removed take.task
removed permiss.deal
removed made.clear—in
removed fiction.cosi
removed receiv.corrupt
removed town.world
removed shift.take
removed said.think
removed matter.someth
removed cadet

removed murder.could
removed quarter.netherton
removed justic.hold
removed age.make
removed behest.ex-pat
removed gerald.howarth
removed british.alli
removed okay.first
removed extend.intervent
removed way.spend
removed lead.opposit
removed duti.scrutinis
removed given.refug
removed far.intellig
removed right.need
removed £59.billion
removed busi.may
removed organis.chosen
removed recognit.qualif
removed colchest.north
removed togeth.marri
removed key.improv
removed disabl.child
removed sight.statement
removed agreement.iran
removed minist.share
removed wrong.voic
removed long.way
removed obstacl.format
removed humanitarian.consequ
removed year.particularli
removed shipley.slightli
removed done.tackl
removed totalitarian.theori
removed oper.saw
removed charity—i.argu
removed slow.part
removed daesh.effect
removed seem.rememb
removed assur.give
removed avoid.lengthi
removed role.school
removed role.capac
removed presid.state
removed impuls.polit
removed visit.hous
removed support.fledgl

removed biggest.problem
removed creed.fascist
removed nation.subject
removed insuffici.number
removed unjustifi.adventur
removed hous.elect
removed syrian.open
removed long.continu
removed price.roam
removed wed.bureaucrat
removed reward.endeavour
removed child.develop
removed fundrais.tactic
removed fled.countri
removed emot.crave
removed think.rather
removed import.nato
removed larkin.poem
removed would.invad
removed world.let
removed europ.pursuit
removed holubchi.varenyki
removed state.fund
removed way.achiev
removed duck.respons
removed within.depart
removed restrict.understand
removed key.point
removed syrian.arab
removed franc.usa
removed extrem.preserv
removed societi.trade
removed pupil.forget
removed armi.forc
removed prescrib.minimum
removed gener.life-sav
removed seem.clear
removed wish.mention
removed convent.crimin
removed capabl.get
removed local.base
removed polish.nation
removed minut.drop
removed commit.refuge
removed candid.contain
removed safer.secur
removed control

removed someon.qualifi
removed award.point
removed neighbour.opportun
removed eventu.came
removed tri.recreat
removed actual.bill
removed howev.majesti
removed region.look
removed daili.agenda
removed success.britain
removed need.use
removed octob.sweden
removed pain.particularli
removed alli.facilit
removed expert.believ
removed enough.cash
removed court.right
removed second.third
removed individu.shown
removed around.math
removed sovereign.state
removed engag.much
removed refer.appal
removed leagu.champion
removed highest.like
removed conflict.greek
removed evil.take
removed maraud.migrant
removed must.differ
removed three.tornado
removed lead.prison
removed sand.tunisia
removed oppos.bomb
removed complet.safe
removed deficit.£27.3
removed less.give
removed point.explain
removed debat.anoth
removed govern.european
removed remov.centr
removed programm.reduc
removed time.yesterday
removed uk.cours
removed done.share
removed bring.polit
removed would.understand
removed spend.level
remov

removed explain.want
removed prove.allus
removed support.malawi
removed get.uk
removed may.afterward
removed time.minist
removed death.destruct
removed travel.throughout
removed horrif.civil
removed still.lone
removed alloc.educ
removed far.brutal
removed team.phillip
removed long.hon
removed constern.kigali
removed clearli.opinion
removed evenli.proportion
removed forc.withdraw
removed precis.design
removed decemb.intern
removed princip.caus
removed first.woman
removed audienc.effect
removed infrastructur.project
removed judg.prosecutor
removed aid.rather
removed problem.sometim
removed live.memori
removed achiev.intervent
removed take.person
removed enabl.govern
removed greec.risk
removed revis.free
removed dictat.idi
removed realli.hypocrisi
removed broken.law
removed large-scal.organis
removed new.govern
removed proud.social
removed rightli.rais
removed independ.eventu
removed method.recruit
removed recent.event
removed overse.british
removed state.like
removed debat.would
removed 

In [214]:
print(len(politicians))
print(len(cols))

109
7


In [215]:
['bigrams'].extend(politicians)

In [216]:
cols = ['bigrams']
cols.extend(politicians)
rows = []
for key in count_bigrams_by_politicians.keys():
    row = [key]
    for politician in politicians:
        row.append(count_bigrams_by_politicians[key][politician])
    rows.append(row)
res_df = pd.DataFrame(rows, columns=cols)
res_df

,bigrams,Kevin Brennan,Rehman Chishti,Peter Bone,Mike Gapes,Jim Cunningham,Henry Smith,Keith Vaz,David Burrowes,Andrew Bridgen,...,Caroline Flint,Charlotte Leslie,Marcus Fysh,Nick Thomas-Symonds,Fiona Mactaggart,Angela Rayner,Rupa Huq,Kevan Jones,Nigel Adams,Andy Burnham
0,first.aid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,govern.take,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,econom.migrant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,may.ask,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,friend.foreign,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,achiev.aim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
940,know.mani,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
941,million.year,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
942,agre.hon,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [217]:
politicians_chosen

,full name,party
0,Kevin Brennan,1
1,Rehman Chishti,0
2,Peter Bone,0
3,Mike Gapes,1
4,Jim Cunningham,1
...,...,...
104,Angela Rayner,1
105,Rupa Huq,1
106,Kevan Jones,1
107,Nigel Adams,0


In [218]:
def calculateChiSq(politicians_chosen, political_Ngrams_20, all_=False, equal_conservative_labour_Ngrams=True, topic=""):

    # we give a file with full list of politicians but choose those that have party info below either way
    lc_names = list(politicians_chosen.loc[politicians_chosen["party"] == 0]["full name"])
    rl_names = list(politicians_chosen.loc[politicians_chosen["party"] == 1]["full name"])

    # we throw away politicians without party info so we delete bigrams that are spoken
    # by none of the politicians that are left to avoid division by zero
    politicians_with_party_info = list(politicians_chosen.dropna(subset=["party"])["full name"].values)
    political_Ngrams_20 = political_Ngrams_20.loc[
                          (political_Ngrams_20[politicians_with_party_info].sum(axis=1) > 0), :]

    C_bc = political_Ngrams_20[lc_names]
    C_bl = political_Ngrams_20[rl_names]

    C_bc = C_bc.sum(axis=1)
    C_bl = C_bl.sum(axis=1)

    C_c = C_bc.sum(axis=0)
    C_l = C_bl.sum(axis=0)

    is_conservative = C_bc.values >= C_bl.values

    print("Total leave_conservative bigrams: " + str(C_c))
    print("Total remain_labour bigrams: " + str(C_l))

    C_nbc = C_c - C_bc
    C_nbl = C_l - C_bl

    den = ((C_bc + C_bl) * (C_bc + C_nbc) * (C_bl + C_nbl) * (C_nbc + C_nbl))
    coeffs = (C_bc * C_nbl - C_bl * C_nbc) ** 2 / den

    Ngrams = list(political_Ngrams_20['bigrams'])
    coeffs = pd.DataFrame(
        {'bigrams': Ngrams, "chi sq": list(coeffs), "leave_conservative": list(is_conservative)})

    print("Sorting values.", flush=True)
    if equal_conservative_labour_Ngrams:
        coeffs_conservative = coeffs.loc[coeffs["leave_conservative"] == True, :]
        coeffs_labour = coeffs.loc[coeffs["leave_conservative"] == False, :]

        coeffs_conservative.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True,
                                        kind='quicksort', na_position='last')
        coeffs_labour.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True, kind='quicksort',
                                  na_position='last')

        if topic == "":
            take_no_Ngrams = 600
        else:
            take_no_Ngrams = 300

        coeffs_conservative = coeffs_conservative.iloc[:take_no_Ngrams, :]
        coeffs_labour = coeffs_labour.iloc[:take_no_Ngrams, :]
        coeffs_conservative_Ngrams = coeffs_conservative['bigrams'].values
        coeffs_labour_Ngrams = coeffs_labour['bigrams'].values
        total_Ngrams = list(coeffs_conservative_Ngrams) + list(coeffs_labour_Ngrams)

        coeffs = coeffs.loc[coeffs['bigrams'].isin(total_Ngrams), :]
        coeffs.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True, kind='quicksort',
                           na_position='last')

    else:
        coeffs.sort_values(by=['chi sq', 'bigrams'], axis=0, ascending=False, inplace=True, kind='quicksort',
                           na_position='last')
        coeffs.to_csv('./all_bigrams.csv')
        if not all_:
            coeffs = coeffs.iloc[:1000, :]

    return coeffs.reset_index(drop=True)

In [219]:
coeffs = calculateChiSq(politicians_chosen,  res_df)

Total leave_conservative bigrams: 7061
Total remain_labour bigrams: 3737
Sorting values.


/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [220]:
coeffs

,bigrams,chi sq,leave_conservative
0,hon.friend,0.012245,True
1,foreign.nation,0.006172,True
2,secretari.state,0.005407,False
3,prime.minist,0.004894,False
4,nation.offend,0.003966,True
...,...,...,...
850,issu.debat,0.000006,True
851,hour.week,0.000006,True
852,hope.minist,0.000006,True
853,good.thing,0.000006,True


In [221]:
coeffs.to_csv('leave_conservative_and_remain_labour_coeffs.csv', index=False)

In [222]:
top_bigrams = coeffs['bigrams'].values

In [223]:
res_df_1000 = res_df.loc[res_df['bigrams'].isin(top_bigrams)].reset_index(drop=True)
res_df_1000

,bigrams,Kevin Brennan,Rehman Chishti,Peter Bone,Mike Gapes,Jim Cunningham,Henry Smith,Keith Vaz,David Burrowes,Andrew Bridgen,...,Caroline Flint,Charlotte Leslie,Marcus Fysh,Nick Thomas-Symonds,Fiona Mactaggart,Angela Rayner,Rupa Huq,Kevan Jones,Nigel Adams,Andy Burnham
0,first.aid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,govern.take,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,econom.migrant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,may.ask,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,friend.foreign,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,achiev.aim,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
851,know.mani,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
852,million.year,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
853,agre.hon,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [224]:
res_df_1000.to_csv('raw_shares.csv', index=False)

In [225]:
res_df.drop(['bigrams'], axis=1).sum(axis=0)

Kevin Brennan      23
Rehman Chishti    127
Peter Bone         56
Mike Gapes        140
Jim Cunningham     63
                 ... 
Angela Rayner      13
Rupa Huq           41
Kevan Jones        29
Nigel Adams         8
Andy Burnham      189
Length: 109, dtype: int64

In [226]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV

def standardize(political_Ngrams_1000):
    Ngrams = political_Ngrams_1000['bigrams']
    col_names = political_Ngrams_1000.columns
    
    political_Ngrams_1000_politicians_only = political_Ngrams_1000.drop(['bigrams'], axis=1).fillna(0)
    
    political_Ngrams_1000_std = political_Ngrams_1000_politicians_only / political_Ngrams_1000_politicians_only.sum(axis=0)
    std = np.std(political_Ngrams_1000_std, axis=1)
    mean = np.mean(political_Ngrams_1000_std, axis=1)
    political_Ngrams_1000_std = (political_Ngrams_1000_std.T - np.mean(political_Ngrams_1000_std, axis=1)).T
    
    political_Ngrams_1000_std = (political_Ngrams_1000_std.T / std).T
    political_Ngrams_1000_std = pd.DataFrame(data=np.column_stack(
    (Ngrams, political_Ngrams_1000_std)), columns=col_names)
    
    mean_std = pd.DataFrame(data=np.column_stack(
    (Ngrams, mean, std)), columns=['bigrams', 'mean', 'std'])
    
    return political_Ngrams_1000_std, mean_std

def calculateBetas(political_Ngrams_1000, political_Ngrams_1000_std, save_alphas_betas=True):

    regressor = LinearRegression(n_jobs=1)
    regressor_no_intercept = LinearRegression(fit_intercept=False, n_jobs=1)

    def regress(X, political_Ngrams_1000, political_Ngrams_1000_std, name=""):

        # remove bigrams with zero row sum for politicians and remove sources with column sum zero that will
        # come to existance after bigrams removal
        political_Ngrams_1000_nonzero_bigrams = list(political_Ngrams_1000.loc[ \
                                                                 (political_Ngrams_1000.drop(
                                                                     'bigrams', axis=1).sum(axis=1) > 0),
                                                                 :]['bigrams'].values)

        political_Ngrams_1000 = political_Ngrams_1000.loc[political_Ngrams_1000['bigrams'].isin(
            political_Ngrams_1000_nonzero_bigrams), :]

        political_Ngrams_1000_std = political_Ngrams_1000_std.loc[political_Ngrams_1000_std[ \
                                                    'bigrams'].isin(
                                                    political_Ngrams_1000_nonzero_bigrams), :]

        political_Ngrams_1000_std = political_Ngrams_1000_std.loc[
                                    political_Ngrams_1000_std['bigrams'].isin(
                                        political_Ngrams_1000_nonzero_bigrams), :]

        # calculate Linear Regression
        Ngrams = list(political_Ngrams_1000_std['bigrams'].values)
        political_Ngrams_1000_std.drop('bigrams', axis=1, inplace=True)

        betas = []
        alphas = []
        for i in range(political_Ngrams_1000_std.shape[0]):
            y = np.array(political_Ngrams_1000_std.iloc[i, :]).ravel()
            regressor.fit(X, y)
            betas.append(regressor.coef_[0])
            alphas.append(regressor.intercept_)

        alphas = np.array(alphas)
        betas = np.array(betas)

        # Save partial information
        if (save_alphas_betas == True):
            final_df = political_Ngrams_1000_std
            final_df.to_csv("./final_dataframe_" + name + "_" + "std_after" + ".csv", index=False)

            alphas_betas_df = pd.DataFrame({'bigrams': Ngrams, "alphas": list(alphas), \
                                            "betas": list(betas),
                                            })
            alphas_betas_df.to_csv("./alphas_betas_" + name + "_" + "std_after" + ".csv", index=False)

        return alphas, betas
    
    X = np.array(politicians_chosen.dropna(subset=["party"])["party"]).reshape(-1, 1)
    politician_final_names = list(politicians_chosen.dropna(subset=["party"])["full name"].values)
    alphas_a, betas_a = regress(X, political_Ngrams_1000, political_Ngrams_1000_std, "party")

    regression_parameters = {"alphas_party": alphas_a, "betas_party": betas_a}

    return regression_parameters

def chooseNames(political_Ngrams_1000_std):
    
    global politicians_chosen
    politicians = pd.read_csv("./brexit_full_updated3.csv")

    politicians = politicians[["full name", "party", "conShare", "conShare2"]]
    politicians = politicians.replace("NA", np.nan)

    # drop politicians that do not have information about "party" and "conShare" and "conShare2" from info dataframe
    politicians = politicians.dropna(subset=["party"], how="all")

    common_names = political_Ngrams_1000_std.columns[1:]

    politicians_chosen = politicians_chosen.loc[politicians_chosen["full name"].isin(common_names)]
    
    print(len(common_names))
    return politicians_chosen

In [227]:
len(cols)

110

In [228]:
res_df_1000 = res_df_1000.loc[:, [True] + list(
            (res_df_1000.drop('bigrams', axis=1).sum(axis=0) > 0).values)]

political_Ngrams_1000_std, std_params = standardize(res_df_1000)
politicians_chosen = chooseNames(political_Ngrams_1000_std).drop_duplicates(subset=['full name'])
cols = political_Ngrams_1000_std.columns[1:]

names = politicians_chosen['full name'].values
cols = [col for col in cols if col in names]
columns = ['bigrams']
columns.extend(cols)
# politicians_chosen2 = politicians_chosen2.loc[politicians_chosen2['full name'].isin(cols)]
political_Ngrams_1000_std = political_Ngrams_1000_std[columns]
display(political_Ngrams_1000_std)
res_df_1000 = res_df_1000[columns]
# display(politicians_chosen)
print("Calculating Betas.")

regression_parameters = calculateBetas(res_df_1000, political_Ngrams_1000_std, save_alphas_betas=True)

109


,bigrams,Kevin Brennan,Rehman Chishti,Peter Bone,Mike Gapes,Jim Cunningham,Henry Smith,Keith Vaz,David Burrowes,Andrew Bridgen,...,Caroline Flint,Charlotte Leslie,Marcus Fysh,Nick Thomas-Symonds,Fiona Mactaggart,Angela Rayner,Rupa Huq,Kevan Jones,Nigel Adams,Andy Burnham
0,first.aid,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,...,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642,-0.268642
1,govern.take,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,...,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529,-0.232529
2,econom.migrant,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,...,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295,-0.160295
3,may.ask,-0.27791,1.0511,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,...,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791,-0.27791
4,friend.foreign,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578,4.81799,-0.159578,-0.159578,-0.159578,...,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578,-0.159578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,achiev.aim,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,...,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475,-0.151475
851,know.mani,-0.222813,-0.222813,-0.222813,-0.222813,-0.222813,0.530587,-0.222813,-0.222813,-0.222813,...,-0.222813,-0.222813,-0.222813,-0.222813,8.06458,-0.222813,-0.222813,-0.222813,-0.222813,-0.222813
852,million.year,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,...,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548,-0.167548
853,agre.hon,-0.233784,5.8469,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,...,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784,-0.233784


Calculating Betas.


In [229]:
regression_parameters

{'alphas_party': array([ 6.43270089e-02, -8.38923977e-02,  1.05400734e-01, -5.45752859e-02,
         1.88301816e-01, -4.26020442e-02, -1.21295843e-01,  1.36725807e-01,
        -1.21339655e-01, -4.56318225e-04, -5.34402725e-02, -2.90488800e-02,
         1.53539392e-01,  1.48202129e-01, -7.37657757e-02,  2.12842118e-01,
         1.23326048e-01,  1.46380204e-01, -2.47248607e-02, -1.36713683e-01,
        -9.10070458e-02,  2.07227654e-01, -8.41033511e-02,  7.75573039e-02,
         1.54350763e-01, -3.15504205e-02, -1.53568412e-02, -1.60197290e-01,
        -1.83384324e-01,  1.76534099e-02, -1.78948200e-02,  4.20002160e-02,
         2.18497703e-01,  5.04407048e-02, -8.39066745e-02, -6.37188140e-02,
         8.00293058e-02, -1.29056188e-02, -9.76852182e-02, -1.22416083e-01,
         4.23692923e-02,  2.03139658e-01,  7.65882198e-02, -5.88039584e-02,
        -6.78236766e-02,  1.19725051e-01, -1.52209483e-01,  9.64872942e-02,
         8.51539091e-02, -7.70868650e-02,  7.71032283e-03, -1.41789687e-

In [230]:
alphas = regression_parameters['alphas_party']
betas = regression_parameters['betas_party']
top_bigrams = res_df_1000['bigrams'].values

In [231]:
regressor_no_intercept = LinearRegression(fit_intercept=False, n_jobs=1)

def predict_affiliations(text, num=0):
    bigrams = get_bigrams(text)
    counts = [bigrams.count(top_bigram) for top_bigram in top_bigrams]
    df = pd.DataFrame(top_bigrams, columns=['bigrams'])
    df['counts'] = counts
    res_df_1000['text{}'.format(num)] = counts
    df_counts_only = df.drop('bigrams', axis=1)
    df_counts_only = ((df_counts_only.T - std_params['mean']) / std_params['std']).T
    political_Ngrams_1000_std['text{}'.format(num)] = df_counts_only['counts']
    df = pd.DataFrame(data=np.column_stack((top_bigrams, df_counts_only)), columns=['bigrams', 'counts'])
#     df = df.fillna(0)
    X = betas.reshape(-1, 1)
    y = np.array((np.array(df['counts']) - alphas)).ravel()
    regressor_no_intercept.fit(X, y)
    
    return regressor_no_intercept.coef_[0]

In [232]:
test_df = pd.read_csv('test_get_the_facts.csv')
test_df

,text
0,Over 3 million UK jobs are linked to our trade...
1,"In the EU you can find work, holiday and retir..."
2,If we leave the EU experts predict that the ec...
3,The Trades Union Congress says the average wag...
4,Being in the EU means you pay less for your we...
5,It's estimated that saves your family £350 a y...
6,The Governor of the Bank of England says Brexi...
7,9 out of 10 economic experts say our economy w...
8,Overall we get more out than we pay in. Britai...
9,The UK economy benefits from investment worth ...


In [233]:
text = ' '.join(test_df['text'].values)

In [234]:
predict_affiliations(text)

-23.07162155728173

In [235]:
std_params

,bigrams,mean,std
0,first.aid,0.007597,0.0282793
1,govern.take,0.00187173,0.00804945
2,econom.migrant,0.00106335,0.00663372
3,may.ask,0.00168638,0.00606808
4,friend.foreign,0.000182156,0.00114149
...,...,...,...
850,achiev.aim,0.00107401,0.00709033
851,know.mani,0.00168036,0.00754157
852,million.year,0.000143255,0.000855006
853,agre.hon,0.000310057,0.00132625


In [236]:
test_df = pd.read_csv('test_key_speeches_combined.csv')
test_df

,text
0,We all agree that it is vital that everyone is...
1,"If we Vote Leave on 23 June, the Government wi..."
2,Thirteen Government ministers and senior Conse...
3,Michael Gove and Dominic Raab have today made ...
4,"Michael Gove, Boris Johnson, Gisela Stuart and..."
5,"Michael Gove, Boris Johnson, and Gisela Stuart..."
6,A STRONGER NHS AND MORE MONEY FOR THOSE IN NEE...
7,Liam Fox MP delivered a speech today at Vote L...
8,"Statement by Michael Gove, Boris Johnson, Prit..."
9,"On 23 June, Sun readers can decide whether we ..."


In [237]:
rows = []

n = len(test_df)
for index in tqdm(range(n)):
    row = test_df.iloc[index]
    transcript = row['text']
#     speaker = row['name']
    rows.append([transcript, predict_affiliations(transcript, index)])
    
aff_df = pd.DataFrame(rows, columns=['transcript', 'affiliations'])
aff_df

100%|██████████| 24/24 [00:10<00:00,  2.29it/s]


,transcript,affiliations
0,We all agree that it is vital that everyone is...,-8.495689
1,"If we Vote Leave on 23 June, the Government wi...",-203.091641
2,Thirteen Government ministers and senior Conse...,-53.371487
3,Michael Gove and Dominic Raab have today made ...,-146.285470
4,"Michael Gove, Boris Johnson, Gisela Stuart and...",-171.848000
5,"Michael Gove, Boris Johnson, and Gisela Stuart...",-47.760025
6,A STRONGER NHS AND MORE MONEY FOR THOSE IN NEE...,-75.661082
7,Liam Fox MP delivered a speech today at Vote L...,-25.488996
8,"Statement by Michael Gove, Boris Johnson, Prit...",-123.198597
9,"On 23 June, Sun readers can decide whether we ...",-42.488201


In [238]:
aff_df.to_csv('key_speeches_prediction.csv', index=False)

In [239]:
political_Ngrams_1000_std.to_csv('final_dataframe_party_std_after.csv', index=False)

In [240]:
res_df_1000.to_csv('raw_bigrams_counts.csv', index=False)